In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv (r'Capstone_data_Jan19_Nov19.csv')  # 181559 rows × 642 columns
readme = pd.read_csv(r'data_type.csv')

C:\Users\shaow\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (59,66,85,95,512,513,514,515,516,517,518,519,520,521,522,529,531,536,539) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
readme_type = readme[['Column Name', 'Type']]
readme_numerical = readme_type[readme_type['Type'] == 'Numerical']
readme_categorical = readme_type[readme_type['Type'] == 'Categorical']
num_data = data._get_numeric_data()
num_cols = list(num_data.columns)
removed_vars = {'X640', 'X641', 'X642','X101'} # X640, x641, x642, X101 are not numericals, excluded
num_cols = [ele for ele in num_cols if ele not in removed_vars]

var_cols = [ele for ele in data.columns if ele not in removed_vars]
cat_cols = list(set(data.columns) - set(num_cols) )

readme_num_list = readme_numerical['Column Name'].to_list()

# check difference between two lists: Find out which columns are marked as numerical variables but should be categorical
cat_marked_as_num = list(set(num_cols) - set(readme_num_list))
cat_marked_as_num.sort()

data[cat_marked_as_num] = data[cat_marked_as_num].astype('category')

**Starting from here, the code is different from other notebooks**

In [4]:
text_variables = data.select_dtypes(include='object')  # we assume all object columns are text columns
text_vars = text_variables.columns.values.tolist()
text_vars.remove('X101')  # we remove X101 from the text columns since it is an ID feature
data_text = data.loc[:, text_vars]  # pick text columns from the original dataset
# data_text = pd.concat([data_text, data['X642']], axis = 1)

In [5]:
na_unknown_combine = []
disease_col = []
onehot_with_na = []
onehot_without_na = []

# Process X87: In this column we have 'positive', 'negative', '1' and '0'; we change 1 to 'positive', 0 to 'negative'
data_text.loc[(data_text.X87 == '0001.000'),'X87'] = 'POSITIVE OR REACTIVE'
data_text.loc[(data_text.X87 == '0000.000'),'X87'] = 'NEGATIVE'

# X496-X510, X521-X526, X528-X536 are columns with 'yes','no','unknown' and na, we want to combine na with 'unknown' and together form the 'nan_unknown' category
for i in range(496, 511):
    na_unknown_combine.append('X' + str(i))
for i in range(521, 527):
    na_unknown_combine.append('X' + str(i))
for i in range(528, 537):
    na_unknown_combine.append('X' + str(i))
for ele in na_unknown_combine:
    data_text[ele] = data_text[ele].where(data_text[ele]!='UNKNOWN','na_or_unknown')
    data_text[ele] = data_text[ele].where(data_text[ele].isna()==False,'na_or_unknown')
'''
# double check if successfully converted na and 'unknown' to 'na_or_unknown'
for ele in na_unknown_combine:
    print(data_text[ele].unique())
'''
for j in range(538, 564):  # X538-X563 are columns contain diseases, the SPECIAL columns!!!
    disease_col.append('X' + str(j))

# we include columns that could be easily one hot encoded into the one_hot_col vector
one_hot_col = [ele for ele in text_vars if ele not in disease_col]  # as long as it is not a disease column, it is one hot encoded

# when we use pd.get_dummies, we need to separate columns that contain na values from columns that do not contain na values
# therefore creat two lists, one stores column(with na values) ids, one stores column(without na values) ids
for ele in one_hot_col:
    if data_text[ele].isna().values.any():
        onehot_with_na.append(ele)
    else:
        onehot_without_na.append(ele)

In [6]:
df1 = pd.get_dummies(data_text[onehot_with_na], dummy_na = True)  # one hot encode text columns with na value
df2 = pd.get_dummies(data_text[onehot_without_na])  # one hot encode text columns without na value

In [7]:
non_text = [ele for ele in data.columns.values.tolist() if ele not in text_vars]

In [8]:
concat_data = pd.concat([data[non_text], df1, df2], axis = 1)
concat_data  # all data except for 'disease' columns

,X640,X641,X642,X1,X2,X3,X4,X5,X6,X7,...,X533_na_or_unknown,X534_NO,X534_YES,X534_na_or_unknown,X535_NO,X535_YES,X535_na_or_unknown,X536_NO,X536_YES,X536_na_or_unknown
0,1,101,0,1.272695,-0.309136,1961,-0.279328,43.891779,157.48,133.000000,...,1,0,0,1,1,0,0,1,0,0
1,2,102,1,1.194891,-0.309136,1962,-0.279328,29.697368,193.04,116.666667,...,1,0,0,1,1,0,0,1,0,0
2,3,103,1,1.817320,-0.309136,1953,-0.279328,29.598352,170.18,125.000000,...,1,0,0,1,1,0,0,1,0,0
3,4,104,0,2.361946,-0.344643,1947,-0.316563,31.321007,170.18,144.666667,...,1,0,0,1,1,0,0,1,0,0
4,5,105,0,1.428302,-0.356715,1959,-0.329222,37.271479,165.10,NaN,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181554,229873,229524,0,1.194891,-0.344643,1962,-0.316563,31.635000,152.40,NaN,...,1,0,0,1,1,0,0,1,0,0
181555,229874,229525,0,0.961480,-0.238121,1965,-0.279328,31.563265,177.80,121.333333,...,1,0,0,1,1,0,0,1,0,0
181556,229875,229526,1,1.350498,-0.362396,1960,-0.335180,31.946982,165.10,NaN,...,1,0,0,1,1,0,0,1,0,0
181557,229876,229527,1,1.039284,-0.309136,1964,-0.279328,28.188906,170.18,110.666667,...,1,0,0,1,1,0,0,1,0,0


In [13]:
# Process disease columns
tmp_disease_df = data_text[disease_col]
tmp_disease_df

,X538,X539,X540,X541,X542,X543,X544,X545,X546,X547,...,X554,X555,X556,X557,X558,X559,X560,X561,X562,X563
0,HIGHBLOODP,HIGHBLOODP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA|NA|NA,IRRPULSE|IRRPULSE|IRRPULSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANCER
2,HIGHBLOODP,HIGHBLOODP,DIABETESDM|DIABETESDM|DIABETESDM|DIABETESDM|DI...,DIABETES|DIABETES|DIABETES|DIABETES|DIABETES|D...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mammogram,NaN,NaN
3,HIGHBLOODP,HIGHBLOODP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DIVERTCLTS,INTDSRDR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DIABETESDM|DIABETESDM|DIABETESDM|DIABETESDM|DI...,DIABETES|DIABETES|DIABETES|DIABETES|DIABETES|D...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,CHRNICPAIN,CHRNICPAIN,NaN,NaN,Spinal Cord Stimulator,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ARTHRITIS2|ARTHRITIS2,ARTHRITIS2|ARTHRITIS2,DEPRESSION,OTHPSYMHD,NaN,NaN,NaN,HRTDSRDR
181555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HRTDSRDR
181556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
dp = tmp_disease_df['X558'].str.split('|').dropna()
dp

14           [ANXIETY]
24              [ADHD]
25            [STRESS]
27            [NA, NA]
36            [NA, NA]
              ...     
181532    [DEPRESSION]
181543       [ANXIETY]
181546    [DEPRESSION]
181549           [ADD]
181554    [DEPRESSION]
Name: X558, Length: 17343, dtype: object

In [44]:
total_disease = set()  # this is a set contains all unique diseases, including free responses
disease_count = {}  # number of occurance of each disease
unique_num_per_col = {}  # number of unique diseases per column
unique_disease_per_col = {}

for ele in tmp_disease_df:  # iterate through each column
    colwise_collection = set()  # to collect unique disease in each set
    
    # drop NaN, each cell becomes a list, may have duplicates
    # Ex: ANXIETY|ANXIETY -> [ANXIETY, ANXIETY]
    dp = tmp_disease_df[ele].str.split('|').dropna()
    for disease_list in dp:  # iterate through the cells of each column
        for disease in disease_list:  # iterate through the diseases in each cell
            if disease != 'NA':  # we do not include NA as a disease
                if disease in disease_count: disease_count[disease] += 1
                else: disease_count[disease] = 1
        col = set(disease_list)  # turn the list into a set to get unique disease types of each cell
        for _ in col:
            if _ != 'NA':
                if _ not in colwise_collection:
                    colwise_collection.add(_)
                if _ not in total_disease:
                    total_disease.add(_)
    unique_num_per_col[ele] = len(colwise_collection)
    unique_disease_per_col[ele] = colwise_collection

In [45]:
unique_disease_per_col

{'X538': {'ANGINA',
  'AORTICANEU',
  'AORTICINSU',
  'AORTICSTEN',
  'ARTERYBLOC',
  'ASDATRIALS',
  'BENIGNFUN',
  'BLOCKEDHEA',
  'CARDIACARR',
  'CARDIOMYOP',
  'CHESTDCMFT',
  'CHESTPAINS',
  'CHESTPRESS',
  'CONGENHRTD',
  'CONGESTHF',
  'CORONARYAR',
  'DIASTOLICM',
  'EDEMA',
  'ELBLDPRESS',
  'ENDOCARDIT',
  'FALLOTSTE',
  'HEARTATTCK',
  'HEARTDSE',
  'HEARTFAILU',
  'HEARTMROTH',
  'HEARTMRUNK',
  'HEARTMURMU',
  'HEARTTRANS',
  'HEARTVALVE',
  'HIGHBLOODP',
  'HOLEINTHEH',
  'HYPERTENSI',
  'HYPERTROPH',
  'INNOCENTMU',
  'LOWBLOODPR',
  'MARFANSYND',
  'MITRALINSU',
  'MITRALVALV',
  'MITVALPRLP',
  'MITVALRGRT',
  'MYOCARDIAL',
  'MYOCARDITI',
  'OTHER3A',
  'OTHERHEART',
  'PERICARDIT',
  'PERIPHVASC',
  'PREECLAMPS',
  'PREGNANCYI',
  'PVDPERIPHE',
  'UNSTABLEAN',
  'VENTRISEPT',
  'VSCLRDSS',
  'VSDVENTRIC'},
 'X539': {'CHESTDCMFT',
  'CHESTPAINS',
  'ELBLDPRESS',
  'HEARTATTCK',
  'HEARTDSE',
  'HEARTTRANS',
  'HIGHBLOODP',
  'IRRPULSE',
  'MURMUR',
  'OTHCARDSR',
  '

In [46]:
unique_num_per_col

{'X538': 53,
 'X539': 12,
 'X540': 14,
 'X541': 4,
 'X542': 37,
 'X543': 7,
 'X544': 64,
 'X545': 6,
 'X546': 61,
 'X547': 10,
 'X548': 74,
 'X549': 10,
 'X550': 17,
 'X551': 3,
 'X552': 36,
 'X553': 5,
 'X554': 52,
 'X555': 7,
 'X556': 92,
 'X557': 7,
 'X558': 36,
 'X559': 2,
 'X560': 4160,
 'X561': 478,
 'X562': 1,
 'X563': 4}

In [47]:
disease_count

{'HIGHBLOODP': 40478,
 'ANGINA': 242,
 'HEARTMURMU': 1655,
 'HYPERTENSI': 1117,
 'HEARTATTCK': 1359,
 'OTHER3A': 958,
 'CONGESTHF': 50,
 'CHESTPAINS': 7049,
 'CHESTPRESS': 45,
 'CORONARYAR': 37,
 'HEARTDSE': 433,
 'ELBLDPRESS': 577,
 'ARTERYBLOC': 137,
 'BLOCKEDHEA': 65,
 'CARDIACARR': 8,
 'OTHERHEART': 876,
 'MYOCARDIAL': 15,
 'MITVALPRLP': 411,
 'BENIGNFUN': 42,
 'PREGNANCYI': 102,
 'HEARTVALVE': 12,
 'MYOCARDITI': 8,
 'INNOCENTMU': 12,
 'EDEMA': 34,
 'FALLOTSTE': 7,
 'PERICARDIT': 30,
 'CHESTDCMFT': 180,
 'LOWBLOODPR': 60,
 'AORTICINSU': 8,
 'PREECLAMPS': 274,
 'HEARTMROTH': 60,
 'HEARTFAILU': 21,
 'MARFANSYND': 5,
 'ASDATRIALS': 8,
 'CARDIOMYOP': 37,
 'MITVALRGRT': 23,
 'UNSTABLEAN': 6,
 'AORTICANEU': 20,
 'HYPERTROPH': 8,
 'AORTICSTEN': 11,
 'ENDOCARDIT': 4,
 'HOLEINTHEH': 6,
 'CONGENHRTD': 8,
 'PERIPHVASC': 2,
 'PVDPERIPHE': 3,
 'VSDVENTRIC': 9,
 'HEARTMRUNK': 9,
 'VSCLRDSS': 3,
 'VENTRISEPT': 3,
 'MITRALVALV': 9,
 'HEARTTRANS': 2,
 'MITRALINSU': 15,
 'DIASTOLICM': 3,
 'IRRPULSE'

In [48]:
total_disease

{'Other - Blood Draw',
 'LEGPAIN',
 'Fracture broken left ankle',
 'Other - HPV Testing',
 'Laser surgery for enlarged prostate',
 'Knee Repair',
 'Right Hand Surgery',
 'SQUAMOUSCE',
 'Ear surgery to open up a blockage in his ears',
 'Adjustment of Teeth/ Hit in Face with Baseball',
 'Knee meniscus',
 'Breast Implant Surgery',
 'Blood Work',
 'Removal of ductal carcinoma in situ',
 'Varicose Vein Surgical Ligation And Stripping',
 'Microfracture Surgery',
 'Bicep tendon repair',
 'Gluteal implant',
 "Surgery for pain in testicle, doesn't recall name of surgery.",
 'OTHENDDIS',
 'Cervical Disc in her neck of the C5 and C6 replaced',
 'NARCOLEPSY',
 'ACL Reconstruciton',
 'Right Eye Surgery',
 'FNA (Fine Needle Aspiration)',
 'Microscopic Carpal Tunnel Surgery',
 'Wisdom tooth extraction',
 'AC Joint Repain and Reconstruction',
 'Meniscus Tear Repair',
 'Elbow, clean out/',
 'Eyelids raised',
 'Meniscus Surgery on right knee',
 'Left bunionectomy',
 'Removal of uterus',
 'Repair torn me

<font size="5">Visualization</font>